# Quantize GRU

In [1]:
from model import PerformanceRNN
import torch
from torch import nn
import distiller
from distiller.modules.gru import convert_model_to_distiller_gru
from tqdm import tqdm
import numpy as np

### Loading the model and converting to our own implementation.

In [ ]:
sum(p.numel() for p in rnn_model.parameters())

In [ ]:
sum(p.numel() for p in rnn_model.parameters() if p.requires_grad)

In [ ]:
print(rnn_model)

In [2]:
assert torch.cuda.is_available()
device = 'cuda:0'
sess_path = "save/ecomp_w500.sess"
state = torch.load(sess_path)
rnn_model = PerformanceRNN(**state['model_config']).to(device)
rnn_model.load_state_dict(state['model_state'])

In [3]:
rnn_model = convert_model_to_distiller_gru(rnn_model)

## Measure model stats

In [ ]:
events

In [ ]:
from distiller.model_summaries import model_summary

model_summary(rnn_model,  'compute', dataset=events)

Check that man_model is on GRU.

In [ ]:
next(man_model.parameters()).is_cuda

### Check that the conversion has succeeded:

In [ ]:
rnn_model.eval()

In [ ]:
man_model.eval()

Test to make sure that both the original and manual models can generate output.

In [ ]:
model = rnn_model#quantizer.model.to(device)
model.eval()
batch_size = 8
init = torch.randn(batch_size, model.init_dim).to(device)
max_len = 1000
controls=None
greedy_ratio = 0.7
temperature = 1.0

import pdb

with torch.no_grad():
    #pdb.set_trace()
    outputs = model.generate(init, max_len,
                             controls=controls,
                             greedy=greedy_ratio,
                             temperature=temperature,
                             verbose=True)
    

outputs = outputs.cpu().numpy().T # [batch, steps]

Convert output to MIDI and save.

In [ ]:
import utils
import os

output_dir = "quantized_output/"
os.makedirs(output_dir, exist_ok=True)

for i, output in enumerate(outputs):
    name = f'output-{i:03d}.mid'
    path = os.path.join(output_dir, name)
    n_notes = utils.event_indeces_to_midi_file(output, path)
    print(f'===> {path} ({n_notes} notes)')

### Defining the evaluation:

# Quantizing the model:

## Collect activation statistics:

The model uses activation statistics to determine how big the quantization range is. The bigger the range - the larger the round off error after quantization which leads to accuracy drop.  
Our goal is to minimize the range s.t. it contains the absolute most of our data.  
After that, we divide the range into chunks of equal size, according to the number of bits, and transform the data according to this scale factor.  
Read more on scale factor calculation [in our docs](https://nervanasystems.github.io/distiller/algo_quantization.html).

The class `QuantCalibrationStatsCollector` collects the statistics for defining the range $r = max - min$.  

Each forward pass, the collector records the values of inputs and outputs, for each layer:
- absolute over all batches min, max (stored in `min`, `max`)
- average over batches, per batch min, max (stored in `avg_min`, `avg_max`)
- mean
- std
- shape of output tensor  

All these values can be used to define the range of quantization, e.g. we can use the absolute `min`, `max` to define the range.

Check that `man_model` has the same weights as `rnn_model` (Warning: Running this will move the models to the CPU).

In [ ]:
import numpy.testing as nptest

man_model_weights = man_model.cpu().output_fc.weight.detach().numpy()
rnn_model_weights = rnn_model.cpu().output_fc.weight.detach().numpy()
nptest.assert_array_almost_equal(man_model_weights, rnn_model_weights)

Check that `man_model` is on the GPU.

In [ ]:
next(man_model.parameters()).is_cuda

In [ ]:
# My version.
import os
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

# Commented line is probably not necessary.
#man_model = torch.load('./manual.checkpoint.pth.tar')
distiller.utils.assign_layer_fq_names(rnn_model)
collector = QuantCalibrationStatsCollector(rnn_model)
                                           #inplace_runtime_check=True)
                                           #disable_inplace_attrs=True) # I added this last argument.

# Random numbers.
batch_size = 64
max_len = 100


with collector_context(collector) as collector:
    init = torch.randn(batch_size, rnn_model.init_dim).to(device)
    output = rnn_model.generate(init, max_len)
    collector.save('performance_rnn_pretrained_stats.yaml')

## Quantize Model:

In [ ]:
overrides_yaml = """
#.*eltwise.*:
#    fp16: true
output_fc_activation:
    fp16: true
"""

In [4]:
from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy
# Define the quantizer
overrides_yaml = """
#.*eltwise.*:
#    fp16: true
output_fc_activation:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(rnn_model),
    model_activation_stats='stats/performance_rnn_pretrained_stats.yaml',
    overrides=overrides,
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True,
    clip_acts='AVG'
)

# Quantizer magic:

quantizer.prepare_model()

In [5]:
quantizer.model

PerformanceRNN(
  (inithid_fc): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True, scale_approx_mult_bits=None
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=61.0203, in_zero_point=2.0000
    out_scale=37.8976, out_zero_point=-16.0000
    (wrapped_module): Linear(in_features=32, out_features=1536, bias=True)
  )
  (inithid_fc_activation): Tanh()
  (event_embedding): RangeLinearEmbeddingWrapper(
    (wrapped_module): Embedding(240, 240)
  )
  (concat_input_fc): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True, scale_approx_mult_bits=None
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=205.7303, in_zero_point=79.0000
    out_scale=145.4613, out_zero_point=-22.0000
    (wrapped_module): Linear(in_features=265, o

In [6]:
quantizer.model.__dict__.keys()

dict_keys(['_backend', '_parameters', '_buffers', '_backward_hooks', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_modules', 'training', 'event_dim', 'control_dim', 'init_dim', 'hidden_dim', 'gru_layers', 'concat_dim', 'input_dim', 'output_dim', 'primary_event', 'quantizer_metadata'])

In [20]:
for key, val in quantizer.model._modules.items():
    print(key, val)

inithid_fc RangeLinearQuantParamLayerWrapper(
  mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True, scale_approx_mult_bits=None
  preset_activation_stats=True
  w_scale=PerCh, w_zero_point=PerCh
  in_scale=61.0203, in_zero_point=2.0000
  out_scale=37.8976, out_zero_point=-16.0000
  (wrapped_module): Linear(in_features=32, out_features=1536, bias=True)
)
inithid_fc_activation Tanh()
event_embedding RangeLinearEmbeddingWrapper(
  (wrapped_module): Embedding(240, 240)
)
concat_input_fc RangeLinearQuantParamLayerWrapper(
  mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True, scale_approx_mult_bits=None
  preset_activation_stats=True
  w_scale=PerCh, w_zero_point=PerCh
  in_scale=205.7303, in_zero_point=79.0000
  out_scale=145.4613, out_zero_point=-22.0000
  (wrapped_module): Linear(in_features=265, out_features=512, bias=True)
)
concat_input_fc_activation LeakyReLU

In [47]:
quantizer.model._modules['gru']._modules['cells'][0]._modules['fc_gate_x'].wrapped_module.__dict__['_parameters']['weight']

Parameter containing:
tensor([[  15.,  -25.,  -26.,  ...,  -43.,  -34.,   38.],
        [ -24.,  -45.,  -46.,  ...,  -47.,    4.,   69.],
        [  -7.,   70., -101.,  ...,   58.,   24.,   33.],
        ...,
        [ -27.,   11.,  -57.,  ...,  -21.,  -41.,  -55.],
        [  40.,   27.,    7.,  ...,   76.,   42.,   56.],
        [ -24.,   11.,  -32.,  ...,   19.,  -16.,  -40.]], device='cuda:0',
       requires_grad=True)

In [45]:
quantizer.model._modules['gru']._modules['cells'][0]._modules['eltwiseadd_gate'].wrapped_module

EltwiseAdd()

## Evaluate the perplexity of the original and quantized models.

In [ ]:
from data import Dataset
from sequence import EventSeq
import torch.functional as F

data_path = "dataset/processed/ecomp_piano"
dataset = Dataset(data_path, verbose=True)
dataset_size = len(dataset.samples)
assert dataset_size > 0

# Eventually need to put these in YAML file.
controls = None
teacher_forcing_ratio = 1.0
loss_function = nn.CrossEntropyLoss()
log_softmax = nn.LogSoftmax(dim=1)
nnl = nn.NLLLoss(reduction='sum')




window_size = 200
stride_size = 10
use_transposition = False
control_ratio = 1.0
event_dim = EventSeq.dim()
batch_size = 2

batch_gen = dataset.batches(batch_size, window_size, stride_size)

In [ ]:
model = quantizer.model.to(device)
model.eval()

In [ ]:
import numpy.testing as nptest
import pdb

acc_loss = 0
N = 0
num_iters = 100

for iteration, (events, controls) in enumerate(batch_gen):
    print(iteration)
    if iteration == num_iters:
        break
        
    if use_transposition:
        offset = np.random.choice(np.arange(-6, 6))
        events, controls = utils.transposition(events, controls, offset)

    events = torch.LongTensor(events).to(device)
    assert events.shape[0] == window_size

    if np.random.random() < control_ratio:
        controls = torch.FloatTensor(controls).to(device)
        assert controls.shape[0] == window_size
    else:
        controls = None

    init = torch.randn(batch_size, model.init_dim).to(device)
    outputs = model.generate(init, window_size, events=events[:-1], controls=controls,
                             teacher_forcing_ratio=teacher_forcing_ratio, output_type='logit')
    

    assert outputs.shape[:2] == events.shape[:2]

    
    loss1 = loss_function(outputs.view(-1, event_dim), events.view(-1))
    pred = log_softmax(outputs.view(-1, event_dim))
    n = pred.shape[0]
    loss2 = nnl(pred, events.view(-1))
    acc_loss += loss2
    N += n
    
    #pdb.set_trace()
   
    # Check to make sure we're calculating the correct loss.
    nptest.assert_array_almost_equal(loss1.cpu().detach().numpy(), loss2.cpu().detach().numpy() / n)



acc_cross_entropy_loss = acc_loss / N
perplexity = acc_cross_entropy_loss.exp()
perplexity

# Collect Quant Stats Part II

In [ ]:
from data import Dataset
from sequence import EventSeq
import torch.functional as F

data_path = "dataset/processed/ecomp_piano"
dataset = Dataset(data_path, verbose=True)
dataset_size = len(dataset.samples)
assert dataset_size > 0

# Eventually need to put these in YAML file.
controls = None
teacher_forcing_ratio = 1.0
loss_function = nn.CrossEntropyLoss()
log_softmax = nn.LogSoftmax(dim=1)
nnl = nn.NLLLoss(reduction='sum')




window_size = 200
stride_size = 10
use_transposition = False
control_ratio = 1.0
event_dim = EventSeq.dim()
batch_size = 2

batch_gen = dataset.batches(batch_size, window_size, stride_size)

In [ ]:
model = rnn_model #quantizer.model.to(device)
model.eval()

In [ ]:
import numpy.testing as nptest
import pdb
import os
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

acc_loss = 0
N = 0
num_iters = 100

# Collect stats.
distiller.utils.assign_layer_fq_names(rnn_model)
collector = QuantCalibrationStatsCollector(rnn_model)
                                           #inplace_runtime_check=True)


with collector_context(collector) as collector:


    for iteration, (events, controls) in enumerate(batch_gen):
        print(iteration)

        if iteration == num_iters:
            break

        if use_transposition:
            offset = np.random.choice(np.arange(-6, 6))
            events, controls = utils.transposition(events, controls, offset)

        import pdb
        pdb.set_trace()
            
        events = torch.LongTensor(events).to(device)
        assert events.shape[0] == window_size

        if np.random.random() < control_ratio:
            controls = torch.FloatTensor(controls).to(device)
            assert controls.shape[0] == window_size
        else:
            controls = None

        init = torch.randn(batch_size, model.init_dim).to(device)
        outputs = model.generate(init, window_size, events=events[:-1], controls=controls,
                                 teacher_forcing_ratio=teacher_forcing_ratio, output_type='logit')


        assert outputs.shape[:2] == events.shape[:2]



    # Save stats.
    collector.save('performance_rnn_pretrained_stats.yaml')